In [1]:
import tensorflow as tf
import numpy as np

In [4]:
sample = "Mulmi team, cheer-up"
# "Mulmi team, cheer-u" → "Mulmi team, cheer-up"

In [7]:
idx2char = list(set(sample)) # 중복 제거
char2idx = {c: i for i, c in enumerate(idx2char)}
char2idx

{'M': 0,
 't': 1,
 'e': 2,
 ',': 3,
 'm': 4,
 ' ': 5,
 'r': 6,
 'i': 7,
 'u': 8,
 'h': 9,
 '-': 10,
 'a': 11,
 'p': 12,
 'l': 13,
 'c': 14}

In [9]:
dic_size = len(char2idx) #  원핫인코딩 크기 rnn input size
hidden_size = num_classes = dic_size

In [10]:
hidden_size #  output size (최종 출력 X) hidden cell에서의 출력

15

In [12]:
num_classes # 최종 output size(softmax)

15

In [13]:
batch_size = 1 # 데이터 1개 → batch: 업데이트 크기

In [14]:
sequence_length = len(sample) - 1
learning_rate = 0.1

In [31]:
sample_idx = [char2idx[c] for c in sample]
print(sample_idx)
xdata = [sample_idx[:-1]]
print(x)
ydata = [sample_idx[1:]]
print(ydata)

[0, 8, 13, 4, 7, 5, 1, 2, 11, 4, 3, 5, 14, 9, 2, 2, 6, 10, 8, 12]
Tensor("Placeholder:0", shape=(?, 19), dtype=int32)
[[8, 13, 4, 7, 5, 1, 2, 11, 4, 3, 5, 14, 9, 2, 2, 6, 10, 8, 12]]


In [22]:
tf.reset_default_graph()
x = tf.placeholder(tf.int32, [None, sequence_length])
y = tf.placeholder(tf.int32, [None, sequence_length])
x_one_hot = tf.one_hot(x, num_classes)
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
ini_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=ini_state, dtype=tf.float32)

W0822 13:54:52.819814 27424 deprecation.py:506] From C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0822 13:54:52.829789 27424 deprecation.py:506] From C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\ops\rnn_cell_impl.py:738: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [39]:
xfc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(xfc, num_classes, activation_fn=None)
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
prediction = tf.argmax(outputs, 2)

In [50]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(150):
        cv, _ = sess.run([sequence_loss, train], feed_dict={x: xdata, y: ydata})
        res = sess.run(prediction, feed_dict={x: xdata})
        res_str = [idx2char[c] for c in np.squeeze(res)]
        print(i, "loss: ", cv, "prediction", "".join(res_str))

0 loss:  2.7110074 prediction uuuuuuuuuuuuuuuuuuu
1 loss:  2.657538 prediction eeeeeeeeeeeeeeeeeee
2 loss:  2.6572065 prediction eeeeeeeeeeeeeeeeeee
3 loss:  2.5945394 prediction eeeeeeeeeeeeeeeeeee
4 loss:  2.5843546 prediction eeeeeeeeeeeeeeeeeee
5 loss:  2.5773036 prediction eeeeeeeeeeeeeeeeeee
6 loss:  2.5731893 prediction eeeeeeeeeeeeeeeeeee
7 loss:  2.549724 prediction ueeeeeeeeeeeeeeeeee
8 loss:  2.5239646 prediction uueeeeeeeeeeeeeeeee
9 loss:  2.4943154 prediction uuuueeeeeeeeeeeeeee
10 loss:  2.4078412 prediction uuuuueeeeeeeeeeeeee
11 loss:  2.3283362 prediction mmmeeeeeeeeeeeeeeee
12 loss:  2.2274988 prediction mmmeeeeememeeeeeeme
13 loss:  2.0171623 prediction lllemeeel meeeeemll
14 loss:  2.739367 prediction llueeeeeueeeeeeeeue
15 loss:  2.3072693 prediction uuuehhhhuhhhhhhhh h
16 loss:  2.2964926 prediction uuueeeeeueeeeeeeeue
17 loss:  2.0167599 prediction uuu  eeeue eeeeeeue
18 loss:  1.9671414 prediction uuuuu eeu ueeeee ue
19 loss:  2.0579426 prediction uuuuu eeueuee

In [52]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [53]:
dataset = [["사과", "치즈", "생수"],
           ["생수", "호두", "치즈", "고등어"],
           ["수박", "사과", "생수"],
           ["생수", "호두", "치즈", "옥수수"]]

In [56]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
te_ary # true = 구매, Flase = 비구매

array([[False,  True,  True, False, False,  True, False],
       [ True, False,  True, False, False,  True,  True],
       [False,  True,  True,  True, False, False, False],
       [False, False,  True, False,  True,  True,  True]])

In [60]:
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,고등어,사과,생수,수박,옥수수,치즈,호두
0,False,True,True,False,False,True,False
1,True,False,True,False,False,True,True
2,False,True,True,True,False,False,False
3,False,False,True,False,True,True,True


In [62]:
freq_item = apriori(df, min_support = 0.5, use_colnames=True)
freq_item

,support,itemsets
0,0.50,(사과)
1,1.00,(생수)
2,0.75,(치즈)
3,0.50,(호두)
4,0.50,"(사과, 생수)"
5,0.75,"(치즈, 생수)"
6,0.50,"(호두, 생수)"
7,0.50,"(호두, 치즈)"
8,0.50,"(호두, 치즈, 생수)"


In [63]:
from mlxtend.frequent_patterns import association_rules

In [65]:
association_rules(freq_item, metric="confidence", min_threshold=0.3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(사과),(생수),0.50,1.00,0.50,1.000000,1.000000,0.000,inf
1,(생수),(사과),1.00,0.50,0.50,0.500000,1.000000,0.000,1.0
2,(치즈),(생수),0.75,1.00,0.75,1.000000,1.000000,0.000,inf
3,(생수),(치즈),1.00,0.75,0.75,0.750000,1.000000,0.000,1.0
4,(호두),(생수),0.50,1.00,0.50,1.000000,1.000000,0.000,inf
5,(생수),(호두),1.00,0.50,0.50,0.500000,1.000000,0.000,1.0
6,(호두),(치즈),0.50,0.75,0.50,1.000000,1.333333,0.125,inf
7,(치즈),(호두),0.75,0.50,0.50,0.666667,1.333333,0.125,1.5
8,"(호두, 치즈)",(생수),0.50,1.00,0.50,1.000000,1.000000,0.000,inf
9,"(호두, 생수)",(치즈),0.50,0.75,0.50,1.000000,1.333333,0.125,inf
